In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data


In [2]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#每个批次的大小
batch_size = 50
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#参数摘要
def variable_summary(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean) #平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram',var)

#f命名空间
with tf.name_scope('input_f'):
    #定义f的两个placeholder
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,10])

#g命名空间
with tf.name_scope('input_g'):
    #定义的g的两个placeholder
    A_x = tf.placeholder(tf.float32,[None,784])
    A_y = tf.placeholder(tf.float32,[None,1])

In [4]:
#创建f的神经网络
with tf.name_scope('layer1_f_w'): 
    W1 = tf.Variable(tf.truncated_normal([784,2000],stddev = 0.1))
    variable_summary(W1)
with tf.name_scope('layer1_f_b'): 
    b1 = tf.Variable(tf.zeros([2000])+0.1)
    variable_summary(b1)
with tf.name_scope('layer1_output'): 
    L1 = tf.nn.relu(tf.matmul(x,W1)+b1)

with tf.name_scope('layer2_f_w'): 
    W2 = tf.Variable(tf.truncated_normal([2000,10],stddev = 0.1))
    variable_summary(W2)
with tf.name_scope('layer2_f_b'): 
    b2 = tf.Variable(tf.zeros([10])+0.1)
    variable_summary(b2)

with tf.name_scope('prediction_f'): 
    prediction = tf.nn.softmax(tf.matmul(L1,W2)+b2)
    


#二次代价函数
#loss = tf.reduce_mean(tf.square(y-prediction))

#cross entropy
with tf.name_scope('loss_f'): 
    loss_f = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = prediction))
    tf.summary.scalar('loss_f',loss_f)
#使用梯度下降法
with tf.name_scope('train_f'): 
    train_step = tf.train.GradientDescentOptimizer(0.02).minimize(loss_f)



#结果存放到一个boolean型列表中
with tf.name_scope('accuracy_f'):
    with tf.name_scope('correct_prediction_f'): 
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在位置
#求准确率
    with tf.name_scope('accuracy_f'): 
        accuracy_f = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy_f',accuracy_f)

In [5]:
#创建一个g(active learning)的神经网络

#权重 偏置
with tf.name_scope('layer1_g_w'): 
    A_W1 = tf.Variable(tf.truncated_normal([784,2700],stddev = 0.1))
    variable_summary(A_W1)
with tf.name_scope('layer1_g_b'): 
    A_b1 = tf.Variable(tf.zeros([2700])+0.1)
    variable_summary(A_b1)
with tf.name_scope('layer1_g_output'): 
    A_L1 = tf.nn.relu(tf.matmul(A_x,A_W1)+A_b1)
    


with tf.name_scope('layer2_g_w'): 
    A_W2 = tf.Variable(tf.truncated_normal([2700,1],stddev = 0.1))
    variable_summary(A_W2)
with tf.name_scope('layer2_g_b'): 
    A_b2 = tf.Variable(tf.zeros([1])+0.1)
    variable_summary(A_b2)
with tf.name_scope('prediction_g'): 
    
    A_prediction = tf.abs(tf.matmul(A_L1,A_W2) + A_b2)

#cross entropy
with tf.name_scope('loss_g'):
#     loss_g = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = A_y, logits = A_prediction))
    loss_g = tf.cast(tf.reduce_mean (tf.square(A_y - A_prediction)), tf.float32)
    tf.summary.scalar('loss_g',loss_g)
    
#使用梯度下降法
with tf.name_scope('train_g'):
    A_train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss_g)

#准确率
with tf.name_scope('accuracy_g'):
    accuracy_g = tf.reduce_mean(A_y - A_prediction)
    tf.summary.scalar('accuracy_g',accuracy_g)


In [6]:
#初始化变量
init = tf.global_variables_initializer()

In [7]:
#模型训练必要的方法
#获取需要query的数据
def get_large_entropy_data_and_label(g_pred, x,y, ent):
    count = 0
    query_data = np.zeros((1,784))
    query_label = np.zeros((1,10))
    query_data_pred_by_g  = np.zeros((1,1))
    for i in range(50):
        if g_pred[i] > ent:
            query_data = np.vstack((query_data,x[i,:]))
            query_label = np.vstack((query_label,y[i,:]))
            query_data_pred_by_g = np.vstack((query_data_pred_by_g,g_pred[i]))
            count = count + 1
        else:
            i = i + 1
    m,n  = np.shape(query_data)
    if m == 1:
        return x, y, 50, g_pred 
        
    else:
        query_data = np.delete(query_data,0,axis = 0)
        query_label = np.delete(query_label,0,axis = 0)
        query_data_pred_by_g = np.delete(query_data_pred_by_g,0,axis = 0)
        print("We have" + " " + str(count))
        return query_data, query_label, count, query_data_pred_by_g 
    


In [8]:

merged_f = tf.summary.merge([tf.get_collection(tf.GraphKeys.SUMMARIES,'layer1_f_w'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer1_f_b'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer1_output'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer2_f_w'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer2_f_b'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'prediction_f'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'loss_f'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'train_f'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'accuracy_f'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'correct_prediction_f')]) 


merged_g = tf.summary.merge([tf.get_collection(tf.GraphKeys.SUMMARIES,'layer1_g_w'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer1_g_b'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer1_g_output'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer2_g_w'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'layer2_g_b'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'prediction_g'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'loss_g'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'train_g'),
                             tf.get_collection(tf.GraphKeys.SUMMARIES,'accuracy_g')]) 
with tf.Session() as sess:
    sess.run(init)
    Query_data_count = 0
    ent_threshold = 0.5
    
    writer_f = tf.summary.FileWriter('trommelNet_f/',sess.graph)
    writer_g = tf.summary.FileWriter('trommelNet_g/',sess.graph)

#先训练一个g    
#     batch_xs,batch_ys = mnist.train.next_batch(batch_size)
#     sess.run(train_step,feed_dict = {x:batch_xs, y:batch_ys})
#     prediction_f = sess.run(prediction, feed_dict = {x:batch_xs})
#     p_index = np.argmax(prediction_f,1)
#     self_entropy = np.zeros([count,1])
#     for i in range(count):          
#         self_entropy[i][0] = - np.log(prediction_f[i][p_index[i]])
#     sess.run(A_train_step, feed_dict = {A_x:batch_xs, A_y: self_entropy})
    
    for epoch in range(1):
        #设置seed pool数量
#         seed_pool_batch = 10
#         for seed_batch in range(seed_pool_batch):
#             batch_xs,batch_ys = mnist.train.next_batch(batch_size)
#             sess.run(train_step,feed_dict = {x:batch_xs, y:batch_ys})
            
        for batch in range(n_batch):


            #batch_xs 50行*784列， batch_ys50行*10列
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)

            #用g先对50个数据预测self entropy 看需不需要query， 生成50行1列的g_prediction
            g_prediction = sess.run(A_prediction,feed_dict = {A_x : batch_xs}) 
            

            #根据g_prediction找出query_data 和 query_label,以及50个传入的数据中，有多少需要query（count）,以及query的数据被g预测的ent
            query_data, query_label, count, query_data_pred_by_g  = get_large_entropy_data_and_label(g_prediction ,batch_xs, batch_ys, ent_threshold)
            Query_data_count = Query_data_count + count

            #用query_data 和 query_label训练f

#             sess.run(train_step,feed_dict = {x:query_data, y:query_label})
            print(np.shape(query_data))
            summary_f,_ = sess.run([merged_f,train_step],feed_dict = {x:query_data, y:query_label})
            prediction_f = sess.run(prediction, feed_dict = {x:query_data})

            #训练g
            #先计算训练所需标签——self entopy
            p_index = np.argmax(prediction_f,1)


            self_entropy = np.zeros([count,1])
            for i in range(count):

                self_entropy[i][0] = - np.log(prediction_f[i][p_index[i]])
                print("Batch" + str(batch) + " " + 
                      "# of image" + str(i) + " " + 
                      "max_pred" + str(prediction_f[i][p_index[i]]) + " " + 
                      "self_ent" + str(self_entropy[i][0]) + " " + 
                      "g_ent" + str(query_data_pred_by_g[i]))
    #             
    #             print(A_loss_g)
    #              test_W_1 = sess.run(A_W1)
            sess.run(A_train_step, feed_dict = {A_x:query_data, A_y: self_entropy})
            A_loss_g = sess.run(loss_g, feed_dict= {A_x:query_data, A_y:self_entropy})
            print("count" + " " + str(count) + " " + str(A_loss_g))
            ent_threshold = ent_threshold +0.0001

            summary_g,_ = sess.run([merged_g, A_train_step], feed_dict = {A_x:query_data, A_y: self_entropy})
            writer_f.add_summary(summary_f,batch)
            writer_g.add_summary(summary_g,batch)
        acc = sess.run(accuracy_f,feed_dict = {x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(epoch) + ", Testing Accuracy" + str(acc) +"using quer" + str(Query_data_count))
    
    


We have 48
(48, 784)
Batch0 # of image0 max_pred0.7323773 self_ent0.311459481716156 g_ent[4.06576872]
Batch0 # of image1 max_pred0.5262851 self_ent0.6419121623039246 g_ent[3.66866732]
Batch0 # of image2 max_pred0.49080998 self_ent0.7116982340812683 g_ent[4.26998425]
Batch0 # of image3 max_pred0.436095 self_ent0.8298951983451843 g_ent[3.24175763]
Batch0 # of image4 max_pred0.53923136 self_ent0.6176105737686157 g_ent[6.20554447]
Batch0 # of image5 max_pred0.4036118 self_ent0.9073017239570618 g_ent[4.44363451]
Batch0 # of image6 max_pred0.4091608 self_ent0.8936470746994019 g_ent[0.80917501]
Batch0 # of image7 max_pred0.5114676 self_ent0.6704710721969604 g_ent[4.47526407]
Batch0 # of image8 max_pred0.43339473 self_ent0.8361063599586487 g_ent[3.36266565]
Batch0 # of image9 max_pred0.37223083 self_ent0.9882411360740662 g_ent[5.27368402]
Batch0 # of image10 max_pred0.64117 self_ent0.44446060061454773 g_ent[2.19487453]
Batch0 # of image11 max_pred0.67882055 self_ent0.38739848136901855 g_ent[2.

We have 50
(50, 784)
Batch2 # of image0 max_pred0.94138 self_ent0.060408368706703186 g_ent[9.73081875]
Batch2 # of image1 max_pred0.3234093 self_ent1.1288366317749023 g_ent[10.60639668]
Batch2 # of image2 max_pred0.2553728 self_ent1.3650308847427368 g_ent[10.7897501]
Batch2 # of image3 max_pred0.65095913 self_ent0.4293084144592285 g_ent[10.92558956]
Batch2 # of image4 max_pred0.6738326 self_ent0.39477357268333435 g_ent[10.16211224]
Batch2 # of image5 max_pred0.47722077 self_ent0.739776074886322 g_ent[10.26969528]
Batch2 # of image6 max_pred0.36484048 self_ent1.0082950592041016 g_ent[10.8887558]
Batch2 # of image7 max_pred0.41367233 self_ent0.8826810717582703 g_ent[10.57824612]
Batch2 # of image8 max_pred0.35739222 self_ent1.0289214849472046 g_ent[10.87816238]
Batch2 # of image9 max_pred0.5557234 self_ent0.5874845385551453 g_ent[10.60198498]
Batch2 # of image10 max_pred0.35903645 self_ent1.0243313312530518 g_ent[10.52731037]
Batch2 # of image11 max_pred0.49205264 self_ent0.7091695666313

We have 50
(50, 784)
Batch4 # of image0 max_pred0.4665283 self_ent0.7624366283416748 g_ent[7.34781361]
Batch4 # of image1 max_pred0.57365453 self_ent0.5557278990745544 g_ent[7.71665955]
Batch4 # of image2 max_pred0.55832565 self_ent0.5828129053115845 g_ent[7.19032192]
Batch4 # of image3 max_pred0.60203254 self_ent0.5074437856674194 g_ent[8.06095505]
Batch4 # of image4 max_pred0.27597347 self_ent1.2874505519866943 g_ent[6.08320379]
Batch4 # of image5 max_pred0.24462788 self_ent1.4080170392990112 g_ent[8.38567638]
Batch4 # of image6 max_pred0.4410408 self_ent0.8186178803443909 g_ent[7.75102806]
Batch4 # of image7 max_pred0.8833568 self_ent0.12402607500553131 g_ent[9.30807686]
Batch4 # of image8 max_pred0.35980374 self_ent1.0221965312957764 g_ent[7.91872835]
Batch4 # of image9 max_pred0.7050003 self_ent0.3495570719242096 g_ent[8.75626278]
Batch4 # of image10 max_pred0.5461008 self_ent0.6049517393112183 g_ent[8.51758862]
Batch4 # of image11 max_pred0.6250665 self_ent0.46989721059799194 g_e

We have 38
(38, 784)
Batch7 # of image0 max_pred0.65145373 self_ent0.4285489022731781 g_ent[5.292346]
Batch7 # of image1 max_pred0.82187235 self_ent0.19617018103599548 g_ent[1.23276711]
Batch7 # of image2 max_pred0.7298261 self_ent0.31494900584220886 g_ent[1.48666573]
Batch7 # of image3 max_pred0.8072892 self_ent0.2140733301639557 g_ent[2.66935825]
Batch7 # of image4 max_pred0.47358114 self_ent0.7474320530891418 g_ent[2.25527287]
Batch7 # of image5 max_pred0.38507378 self_ent0.9543203115463257 g_ent[2.64877415]
Batch7 # of image6 max_pred0.6966611 self_ent0.3614561855792999 g_ent[3.40839672]
Batch7 # of image7 max_pred0.55740196 self_ent0.5844686627388 g_ent[0.79354572]
Batch7 # of image8 max_pred0.47082108 self_ent0.7532771229743958 g_ent[0.55681515]
Batch7 # of image9 max_pred0.37023365 self_ent0.9936209917068481 g_ent[4.21111536]
Batch7 # of image10 max_pred0.4871523 self_ent0.7191784381866455 g_ent[0.97488308]
Batch7 # of image11 max_pred0.38399985 self_ent0.9571130871772766 g_ent[

We have 39
(39, 784)
Batch10 # of image0 max_pred0.3107816 self_ent1.168664813041687 g_ent[1.01265717]
Batch10 # of image1 max_pred0.6653177 self_ent0.4074905812740326 g_ent[2.62998676]
Batch10 # of image2 max_pred0.6661325 self_ent0.4062666594982147 g_ent[5.86132336]
Batch10 # of image3 max_pred0.46277446 self_ent0.770515501499176 g_ent[2.15256119]
Batch10 # of image4 max_pred0.5376866 self_ent0.6204794645309448 g_ent[1.14968777]
Batch10 # of image5 max_pred0.60350204 self_ent0.5050058364868164 g_ent[1.39669228]
Batch10 # of image6 max_pred0.39468548 self_ent0.9296661019325256 g_ent[2.26053429]
Batch10 # of image7 max_pred0.40373477 self_ent0.9069971442222595 g_ent[0.8378334]
Batch10 # of image8 max_pred0.9225271 self_ent0.08063855767250061 g_ent[2.57101631]
Batch10 # of image9 max_pred0.512886 self_ent0.6677017211914062 g_ent[2.42311382]
Batch10 # of image10 max_pred0.6794532 self_ent0.386466920375824 g_ent[1.36408329]
Batch10 # of image11 max_pred0.43171555 self_ent0.839988350868225

...

We have 11
(11, 784)
Batch1048 # of image0 max_pred0.69261307 self_ent0.36728379130363464 g_ent[0.73006725]
Batch1048 # of image1 max_pred0.8367354 self_ent0.1782473474740982 g_ent[1.28621864]
Batch1048 # of image2 max_pred0.99497944 self_ent0.005033204331994057 g_ent[0.84478855]
Batch1048 # of image3 max_pred0.9999876 self_ent1.23978434203309e-05 g_ent[0.81265831]
Batch1048 # of image4 max_pred0.40539443 self_ent0.9028947949409485 g_ent[0.85201454]
Batch1048 # of image5 max_pred0.80345154 self_ent0.2188384085893631 g_ent[0.79576015]
Batch1048 # of image6 max_pred0.9973283 self_ent0.002675293944776058 g_ent[0.67809439]
Batch1048 # of image7 max_pred0.9999709 self_ent2.908749047492165e-05 g_ent[1.19160748]
Batch1048 # of image8 max_pred0.9995735 self_ent0.00042656221194192767 g_ent[0.65073681]
Batch1048 # of image9 max_pred0.99997413 self_ent2.5868750526569784e-05 g_ent[0.64914751]
Batch1048 # of image10 max_pred0.99511236 self_ent0.004899624269455671 g_ent[0.87271023]
count 11
We have 

We have 5
(5, 784)
Batch1057 # of image0 max_pred0.999263 self_ent0.0007372831460088491 g_ent[0.89020538]
Batch1057 # of image1 max_pred0.8260035 self_ent0.1911562830209732 g_ent[0.66107655]
Batch1057 # of image2 max_pred0.98358536 self_ent0.016550855711102486 g_ent[0.78895378]
Batch1057 # of image3 max_pred0.9995925 self_ent0.0004076000186614692 g_ent[0.72706127]
Batch1057 # of image4 max_pred0.94738144 self_ent0.05405348166823387 g_ent[0.69551945]
count 5
We have 5
(5, 784)
Batch1058 # of image0 max_pred0.9895271 self_ent0.010528120212256908 g_ent[0.72891665]
Batch1058 # of image1 max_pred0.94727993 self_ent0.05416063219308853 g_ent[0.62005711]
Batch1058 # of image2 max_pred0.74570537 self_ent0.29342469573020935 g_ent[0.67201757]
Batch1058 # of image3 max_pred0.99365646 self_ent0.0063637495040893555 g_ent[1.32091188]
Batch1058 # of image4 max_pred0.8354298 self_ent0.17980897426605225 g_ent[0.66478014]
count 5
We have 11
(11, 784)
Batch1059 # of image0 max_pred0.7414903 self_ent0.2990

We have 4
(4, 784)
Batch1071 # of image0 max_pred0.9900609 self_ent0.009988796897232533 g_ent[0.61765385]
Batch1071 # of image1 max_pred0.972764 self_ent0.02761375904083252 g_ent[0.66398573]
Batch1071 # of image2 max_pred0.96015143 self_ent0.04066426306962967 g_ent[0.98275328]
Batch1071 # of image3 max_pred0.99467367 self_ent0.005340565927326679 g_ent[0.62180948]
count 4
We have 4
(4, 784)
Batch1072 # of image0 max_pred0.79827726 self_ent0.22529929876327515 g_ent[0.77837801]
Batch1072 # of image1 max_pred0.93814117 self_ent0.06385484337806702 g_ent[0.76668024]
Batch1072 # of image2 max_pred0.9244823 self_ent0.07852138578891754 g_ent[0.95726728]
Batch1072 # of image3 max_pred0.8149261 self_ent0.20465785264968872 g_ent[0.81355429]
count 4
We have 8
(8, 784)
Batch1073 # of image0 max_pred0.8273848 self_ent0.1894853562116623 g_ent[0.65540075]
Batch1073 # of image1 max_pred0.999468 self_ent0.0005321130156517029 g_ent[0.63811111]
Batch1073 # of image2 max_pred0.5899763 self_ent0.527672886848

We have 7
(7, 784)
Batch1082 # of image0 max_pred0.99999976 self_ent2.3841860752327193e-07 g_ent[1.68542385]
Batch1082 # of image1 max_pred0.94124174 self_ent0.06055527552962303 g_ent[1.00622416]
Batch1082 # of image2 max_pred0.95589954 self_ent0.04510245844721794 g_ent[0.68747711]
Batch1082 # of image3 max_pred0.99985933 self_ent0.00014067685697227716 g_ent[0.84299564]
Batch1082 # of image4 max_pred0.99980146 self_ent0.0001985627895919606 g_ent[1.19648457]
Batch1082 # of image5 max_pred0.99247944 self_ent0.007548978552222252 g_ent[0.82953835]
Batch1082 # of image6 max_pred0.9999771 self_ent2.288844552822411e-05 g_ent[0.69023037]
count 7
We have 40
(40, 784)
Batch1083 # of image0 max_pred0.99942756 self_ent0.0005726069211959839 g_ent[1.24109077]
Batch1083 # of image1 max_pred0.997686 self_ent0.002316652797162533 g_ent[0.97652721]
Batch1083 # of image2 max_pred0.99997413 self_ent2.5868750526569784e-05 g_ent[1.71875954]
Batch1083 # of image3 max_pred0.7227094 self_ent0.32474803924560547 

We have 13
(13, 784)
Batch1091 # of image0 max_pred0.7570765 self_ent0.2782909572124481 g_ent[1.23500633]
Batch1091 # of image1 max_pred0.9999211 self_ent7.891966379247606e-05 g_ent[0.62400627]
Batch1091 # of image2 max_pred0.9729793 self_ent0.027392463758587837 g_ent[0.70988607]
Batch1091 # of image3 max_pred0.8377164 self_ent0.17707565426826477 g_ent[0.62995148]
Batch1091 # of image4 max_pred0.9998944 self_ent0.00010562501120148227 g_ent[0.72964287]
Batch1091 # of image5 max_pred0.9795823 self_ent0.02062901295721531 g_ent[0.80074501]
Batch1091 # of image6 max_pred0.9945939 self_ent0.005420747213065624 g_ent[0.63403511]
Batch1091 # of image7 max_pred0.7522487 self_ent0.2846882939338684 g_ent[0.64176273]
Batch1091 # of image8 max_pred0.58814585 self_ent0.5307803153991699 g_ent[0.80192041]
Batch1091 # of image9 max_pred0.98391956 self_ent0.01621113158762455 g_ent[0.73531866]
Batch1091 # of image10 max_pred0.4737516 self_ent0.747072160243988 g_ent[0.62978888]
Batch1091 # of image11 max_p